# Liver Model Construction: Glycolysis Gluconeogenesis & Lactate Fermentation

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.1


## Set paths and constants

### Directory paths

In [6]:
model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

# Create MASS model from COBRA model

### Load COBRA model

In [7]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
core_model

Name,None
Memory address,0x07f7a5e6a0bd0
Number of metabolites,164
Number of reactions,173
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [8]:
glycolysis = MassModel("Glycolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [9]:
glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
                                highlight_missing=True) 
glycolysis_map

ValueError: Could not load map_json. Not valid json, url, or filepath

In [10]:
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L','PYRt2m']
new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 
    
new_reaction_list

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycolysis.add_reactions([MassReaction(reaction)])


### View extracted network

In [12]:
escher_builder = escher.Builder(
    model=glycolysis,
    map_json=os.path.join(
        maps_dir, ".".join((
            glycolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

Builder(highlight_missing=True)

## Parameterize MASS model

# Export MASS model

In [13]:
save_json_mass_model(
    mass_model=glycolysis,
    filename=os.path.join(model_dir, glycolysis.id + ".json"))